In [28]:
#Object Tracking
import cv2
import numpy as np
import time
import math
import sys
#Function to calculate distance between four points
def calDis(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

# Initalize camera
cap = cv2.VideoCapture(0)

# define range of green color in HSV
lower_green = np.array([45,50,50])
upper_green = np.array([75,200,200])
# define range of red color in HSV
lower_red = np.array([0,50,50])
upper_red = np.array([15,200,200])
# define range of blue color in HSV
lower_blue = np.array([105,50,50])
upper_blue = np.array([125,200,200])
#define range of yellow color in HSV
lower_yellow = np.array([22,50,50])
upper_yellow = np.array([32,200,200])

# Create empty points array
yellow = []
green = []
red = []
blue = []

# Get default camera window size
# Here we pass the frame
ret, frame = cap.read()
Height, Width = frame.shape[:2]
frame_count = 0

while True:
    # Capture webcame frame
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blank_image = np.zeros((frame.shape[0], frame.shape[1], 3))
    
    #detection of each color begins
    # Threshold the HSV image to get only green color
    maskg=cv2.inRange(hsv_img, lower_green,upper_green)
    kernelg= np.ones((5,5), np.uint8)
    maskg= cv2.morphologyEx(maskg,cv2.MORPH_OPEN, kernelg)
    contoursg,_ = cv2.findContours(maskg.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Threshold the HSV image to get only yellow color
    masky=cv2.inRange(hsv_img, lower_yellow,upper_yellow)
    kernely= np.ones((5,5), np.uint8)
    masky= cv2.morphologyEx(masky,cv2.MORPH_OPEN, kernely)
    contoursy,_ = cv2.findContours(masky.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Threshold the HSV image to get only blue color
    maskb=cv2.inRange(hsv_img, lower_blue,upper_blue)
    kernelb= np.ones((5,5), np.uint8)
    maskb= cv2.morphologyEx(maskb,cv2.MORPH_OPEN, kernelb)
    contoursb,_ = cv2.findContours(maskb.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Threshold the HSV image to get only red color
    maskr=cv2.inRange(hsv_img, lower_red,upper_red)
    kernelr= np.ones((5,5), np.uint8)
    maskr= cv2.morphologyEx(maskr,cv2.MORPH_OPEN, kernelr)
    contoursr,_ = cv2.findContours(maskr.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    flag = 1
    #Compute after every 10 frames
    if frame_count%10==0:
        if len(contoursg)>0:
            for c in contoursg:
                (x, y), radius = cv2.minEnclosingCircle(c)
                if radius>2:
                    green.append((x,y))
                    cv2.circle(blank_image, (int(x), int(y)), int(radius),(255,255,255), 2)
        else:
            flag=0
        if len(contoursy)>0:
            for c in contoursy:
                (x, y), radius = cv2.minEnclosingCircle(c)
                if radius>2:
                    yellow.append((x,y))
                    cv2.circle(blank_image, (int(x), int(y)), int(radius),(255,255,0), 2)
        else:
            flag=0
        if len(contoursr)>0:
            for c in contoursr:
                (x, y), radius = cv2.minEnclosingCircle(c)
                if radius>2:
                    red.append((x,y))
                    cv2.circle(blank_image, (int(x), int(y)), int(radius),(255,0,0), 2)
        else:
            flag=0
        if len(contoursb)>0:
            for c in contoursb:
                (x, y), radius = cv2.minEnclosingCircle(c)
                if radius>2:
                    blue.append((x,y))
                    cv2.circle(blank_image, (int(x), int(y)), int(radius),(0,0,255), 2)
        else:
            flag=0
        if flag==1:
            for xg,yg in green:
                for xy,yy in yellow:
                    for xr,yr in red:
                        for xb,yb in blue:
                            dis_gy = calDis(xy,xg,yy,yg)
                            dis_gr = calDis(xr,xg,yr,yg)
                            dis_gb = calDis(xb,xg,yb,yg)
                            dis_yr = calDis(xy,xr,yy,yr)
                            dis_yb = calDis(xy,xb,yy,yb)
                            dis_rb = calDis(xr,xb,yr,yb)
                            print(dis_gb,end=' ')
                            print(dis_gr,end=' ')
                            print(dis_gy)
                            if dis_gy<500 and dis_gr<500 and dis_gb<500 and dis_yr<500 and dis_yb<500 and dis_rb<500:
                                print("YES")
                                flag=2
                                cv2.circle(blank_image, (int(xg), int(yg)), int(radius),(255,255,255), -1)
                                cap.release()
                                cv2.destroyAllWindows()
                                sys.exit()
        #if flag!=2:
            #print("NO")
    else:
        green=[]
        red=[]
        yellow=[]
        blue=[]
    frame_count+=1
    blank_image = cv2.flip(blank_image,1)
    cv2.imshow("Object Tracker", blank_image)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

# Release camera and close any open windows
cap.release()
cv2.destroyAllWindows()

152.32328554385566 192.66845439478405 119.77117065001488
YES


SystemExit: 